# Tema lab 4

### Cerinta:
Specificaţi, implementaţi și testați subalgoritmi pentru problema enuntata.

Sa se foloseasca un algoritm de clasificare a imaginilor (etapa de inferenta/testare) si sa se stabileasca performanta acestui algoritm de clasificare binara (imagini cu biciclete vs. imagini fara biciclete).

Pentru imaginile care contin biciclete:

a. sa se localizeze automat bicicletele in aceste imagini si sa se evidentieze chenarele care incadreaza bicicletele

b. sa se eticheteze (fara ajutorul algoritmilor de AI) aceste imagini cu chenare care sa incadreze cat mai exact bicicletele. Care task dureaza mai mult (cel de la punctul a sau cel de la punctul b)?

c. sa se determine performanta algoritmului de la punctul a avand in vedere etichetarile realizate la punctul b (se vor folosi cel putin 2 metrici).

In [18]:
import os
import json
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

In [8]:
# --- SETUP AZURE CREDENTIALS ---
AZURE_KEY = "5i200akRHXqb0YuDuEl7cPbt9p68P3nsm6Lxr28wG6HqqVhjWyHUJQQJ99BCACi5YpzXJ3w3AAAFACOGbqGn"
AZURE_ENDPOINT = "https://alexbalta.cognitiveservices.azure.com/"

client = ComputerVisionClient(AZURE_ENDPOINT, CognitiveServicesCredentials(AZURE_KEY))

## Cerinta 1.

In [9]:
# --- DETECT OBJECTS FUNCTION ---
def detect_objects(image_path):
    with open(image_path, "rb") as img_stream:
        analysis = client.analyze_image_in_stream(img_stream, visual_features=[VisualFeatureTypes.objects])
    return analysis.objects

# --- DRAW BOUNDING BOXES ---
def draw_bounding_boxes(image_path, objects, save_path=None):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    for obj in objects:
        if obj.object_property.lower() == "bicycle":
            rect = obj.rectangle
            draw.rectangle([rect.x, rect.y, rect.x + rect.w, rect.y + rect.h], outline="red", width=3)
            draw.text((rect.x, rect.y - 10), obj.object_property, fill="red")
    if save_path:
        image.save(save_path)
    return image

def classify_image(objects):
    return any(obj.object_property.lower() == "bicycle" for obj in objects)

def process_images(folder_path, output_folder="output"):
    os.makedirs(output_folder, exist_ok=True)
    results = {}
    for filename in os.listdir(folder_path):
        if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
            continue
        path = os.path.join(folder_path, filename)
        objects = detect_objects(path)
        has_bike = classify_image(objects)
        draw_bounding_boxes(path, objects, save_path=os.path.join(output_folder, filename))
        results[filename] = {
            "has_bike": has_bike,
            "objects": [
                {
                    "type": obj.object_property,
                    "rectangle": {
                        "x": obj.rectangle.x,
                        "y": obj.rectangle.y,
                        "w": obj.rectangle.w,
                        "h": obj.rectangle.h
                    }
                }
                for obj in objects
            ]
        }
    with open("results_ai.json", "w") as f:
        json.dump(results, f, indent=2)
    print("Procesare completă. Rezultatele sunt în results_ai.json.")
    return results

Procesare completă. Rezultatele sunt în results_ai.json.


In [ ]:
# --- RULARE (în Jupyter doar apelezi separat dacă vrei) ---
results = process_images("./images")

## Cerinta 2

In [25]:
# Global vars
manual_labels = {}
current_image = None
image_ax = None
clicks = []
image_folder = "./images"
image_list = [f for f in os.listdir(image_folder) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
image_index = 0

def onclick(event):
    global clicks
    if event.inaxes != image_ax:
        return
    x, y = int(event.xdata), int(event.ydata)
    clicks.append((x, y))

    # Draw click
    image_ax.plot(x, y, 'bo')
    fig.canvas.draw()

    # After 2 clicks, save bbox
    if len(clicks) == 2:
        x1, y1 = clicks[0]
        x2, y2 = clicks[1]
        bbox = {
            "x": min(x1, x2),
            "y": min(y1, y2),
            "w": abs(x2 - x1),
            "h": abs(y2 - y1)
        }
        manual_labels[current_image] = [bbox]
        print(f"Etichetare salvată pentru {current_image}: {bbox}")
        clicks.clear()

        # Salvează progresul
        with open("labels_manual.json", "w") as f:
            json.dump(manual_labels, f, indent=2)

        plt.close()

def show_image_for_labeling(index):
    global current_image, fig, image_ax
    current_image = image_list[index]
    path = os.path.join(image_folder, current_image)
    img = Image.open(path)

    fig, image_ax = plt.subplots()
    image_ax.imshow(img)
    image_ax.set_title(f"Etichetare pentru: {current_image}\nClick: colț stânga-sus -> colț dreapta-jos")
    fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show()

In [26]:
%matplotlib notebook

In [39]:
show_image_for_labeling(6) # 0 -> 6

<IPython.core.display.Javascript object>

## Certinta 3

In [35]:
def compute_iou(boxA, boxB):
    xA = max(boxA["x"], boxB["x"])
    yA = max(boxA["y"], boxB["y"])
    xB = min(boxA["x"] + boxA["w"], boxB["x"] + boxB["w"])
    yB = min(boxA["y"] + boxA["h"], boxB["y"] + boxB["h"])

    inter_area = max(0, xB - xA) * max(0, yB - yA)
    boxA_area = boxA["w"] * boxA["h"]
    boxB_area = boxB["w"] * boxB["h"]
    union_area = boxA_area + boxB_area - inter_area

    return inter_area / union_area if union_area > 0 else 0

def evaluate(ai_file="results_ai.json", manual_file="labels_manual.json", iou_threshold=0.5):
    import json

    with open(ai_file) as f:
        ai_data = json.load(f)
    with open(manual_file) as f:
        manual_data = json.load(f)

    TP = FP = FN = 0
    iou_scores = []

    for image in manual_data:
        ai_boxes = [
            obj["rectangle"] for obj in ai_data.get(image, {}).get("objects", [])
            if obj["type"].lower() == "bicycle"
        ]
        gt_box = manual_data[image][0]

        if not ai_boxes:
            FN += 1
            continue

        # Cea mai mare potrivire dintre toate box-urile AI
        best_iou = max([compute_iou(gt_box, pred) for pred in ai_boxes])
        iou_scores.append(best_iou)

        if best_iou >= iou_threshold:
            TP += 1
        else:
            FP += 1

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    avg_iou = sum(iou_scores) / len(iou_scores) if iou_scores else 0

    print(f"📊 Evaluare model AI:")
    print(f"- True Positives (TP): {TP}")
    print(f"- False Positives (FP): {FP}")
    print(f"- False Negatives (FN): {FN}")
    print(f"- Precision: {precision:.2f}")
    print(f"- Recall: {recall:.2f}")
    print(f"- Average IoU: {avg_iou:.2f}")

In [40]:
evaluate()

📊 Evaluare model AI:
- True Positives (TP): 6
- False Positives (FP): 0
- False Negatives (FN): 1
- Precision: 1.00
- Recall: 0.86
- Average IoU: 0.83
